In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
import DataFunctions as data

2024-06-11 09:26:02.425450: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-11 09:26:02.475759: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 09:26:02.475784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 09:26:02.477208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-11 09:26:02.485483: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#Se obtienen los datos de los sensores y las etiquetas deducidas
print("Obteniendo datos...")
data_2206_17_bag, tags_220617_bag = data.get_data('/home/cmonedero/TFG/User1/Motion/User1_220617_Bag.csv', '/home/cmonedero/TFG/User1/220617/Label_New_User1_220617.txt')
# print("Tamaño ventanas X --> ", x_windows_220617_bag.shape)
# print("Tamaño ventanas Y --> ", y_windows_220617_bag.shape)

Obteniendo datos...


Hecho!



In [3]:
x_windows_220617_bag, y_windows_220617_bag = data.divide_data_arrays_into_windows(data_2206_17_bag, tags_220617_bag, window_size=500, moda=True)
x_train_220617_bag, y_train_220617_bag, x_test_220617_bag, y_test_220617_bag = data.divide_train_test(x_windows_220617_bag, y_windows_220617_bag, 0.7)
print(x_windows_220617_bag.shape)
print(y_windows_220617_bag.shape)

Hecho!

(3651, 500, 6)
(3651,)


In [4]:
print(np.unique(y_train_220617_bag))
print(np.unique(y_test_220617_bag))

[1 2 5 6]
[2 3 5]


In [5]:
inputs = Input(shape=(500,6))
x = inputs
print(f'Shape input {inputs.shape}')
x = Conv1D(filters=64, kernel_size=10, data_format='channels_last')(x)
print(f'Shape conv1D {x.shape}')
x = Conv1D(filters=64, kernel_size=10, data_format='channels_last')(x)
print(f'Shape segunda conv1D {x.shape}')
x = MaxPooling1D(pool_size=2)(x)
print(f'Shape despues de Max Pooling {x.shape}')
x = Conv1D(filters=64, kernel_size=10, data_format='channels_last')(x)
print(f'Shape tercera conv1D {x.shape}')
x = Conv1D(filters=64, kernel_size=10, data_format='channels_last')(x)
print(f'Shape cuarta conv1D {x.shape}')
x = GlobalAveragePooling1D(data_format='channels_last')(x)
print(f'Shape despues del global avg pooling {x.shape}')
x = Flatten()(x)
print(f'Shape despues Flatten {x.shape}')
outputs = Dense(8, activation='softmax')(x)
print(f'Output shape {outputs.shape}')

modelo = keras.Model(inputs, outputs)

modelo.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

Shape input (None, 500, 6)
Shape conv1D (None, 491, 64)
Shape segunda conv1D (None, 482, 64)
Shape despues de Max Pooling (None, 241, 64)
Shape tercera conv1D (None, 232, 64)
Shape cuarta conv1D (None, 223, 64)
Shape despues del global avg pooling (None, 64)
Shape despues Flatten (None, 64)
Output shape (None, 8)


2024-05-15 16:08:25.579337: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 970 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:31:00.0, compute capability: 8.6
2024-05-15 16:08:25.581048: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 40933 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:98:00.0, compute capability: 8.6


In [6]:
print(modelo.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500, 6)]          0         
                                                                 
 conv1d (Conv1D)             (None, 491, 64)           3904      
                                                                 
 conv1d_1 (Conv1D)           (None, 482, 64)           41024     
                                                                 
 max_pooling1d (MaxPooling1  (None, 241, 64)           0         
 D)                                                              
                                                                 
 conv1d_2 (Conv1D)           (None, 232, 64)           41024     
                                                                 
 conv1d_3 (Conv1D)           (None, 223, 64)           41024     
                                                             

In [7]:
modelo.fit(tf.constant(x_train_220617_bag), tf.cast(tf.constant(y_train_220617_bag), tf.float32), epochs=10)

Epoch 1/10


2024-05-15 16:08:34.641435: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-05-15 16:08:34.766839: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-15 16:08:35.138093: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f950d296410 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-15 16:08:35.138159: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2024-05-15 16:08:35.138172: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA A40, Compute Capability 8.6
2024-05-15 16:08:35.157204: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-15 16:08:35.267937: I external/local_tsl/tsl/platform/default/subpr

76/80 [===========================>..] - ETA: 0s - loss: 5.0850 - accuracy: 0.5839

2024-05-15 16:08:37.927937: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-15 16:08:37.945675: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-15 16:08:38.013246: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-15 16:08:38.030860: W external/local_tsl/tsl/framework/bfc_

80/80 [==============================] - 5s 18ms/step - loss: 4.8613 - accuracy: 0.5969
Epoch 2/10
80/80 [==============================] - 1s 10ms/step - loss: 0.3364 - accuracy: 0.8912
Epoch 3/10
80/80 [==============================] - 1s 11ms/step - loss: 0.3425 - accuracy: 0.8857
Epoch 4/10
80/80 [==============================] - 1s 11ms/step - loss: 0.4921 - accuracy: 0.8250
Epoch 5/10
80/80 [==============================] - 1s 11ms/step - loss: 0.3317 - accuracy: 0.8806
Epoch 6/10
80/80 [==============================] - 1s 11ms/step - loss: 0.3073 - accuracy: 0.8885
Epoch 7/10
80/80 [==============================] - 1s 11ms/step - loss: 0.3039 - accuracy: 0.8900
Epoch 8/10
80/80 [==============================] - 1s 11ms/step - loss: 0.3178 - accuracy: 0.8748
Epoch 9/10
80/80 [==============================] - 1s 11ms/step - loss: 0.3110 - accuracy: 0.8904
Epoch 10/10
80/80 [==============================] - 1s 11ms/step - loss: 0.3418 - accuracy: 0.8939


In [10]:
evals = modelo.evaluate(tf.constant(x_test_220617_bag), tf.cast(tf.constant(y_test_220617_bag), tf.float32))

 1/35 [..............................] - ETA: 1s - loss: 2.0839e-04 - accuracy: 1.0000

35/35 [==============================] - 0s 5ms/step - loss: 16.6081 - accuracy: 0.6916


In [ ]:
modelo.save('CNNMemo/cnn.h5')